In [7]:
# simulation.py
import bacteria
import grid

import random
import matplotlib.pyplot as plt

def run_simulation(N, Q, NA, NB, NC, F, TS, TR, TF, TA, TRP, TM, K):
    g = grid.create_grid(N)
    species_counts = {'A': NA, 'B': NB, 'C': NC, 'BC': 0}
    all_bacteria = []
    events = []

    # Inicializar bactérias A, B e C na grelha
    for species, count in zip(['A', 'B', 'C'], [NA, NB, NC]):
        for _ in range(count):
            x, y = random.randint(0, N - 1), random.randint(0, N - 1)
            b = bacteria.create_bacteria(species, F)
            b.x, b.y = x, y
            grid.add_bacteria_to_cell(g, x, y, b)
            all_bacteria.append(b)

    # Executar simulação até TS
    t = 0
    while t <= TS:
        for b in all_bacteria:
            # Deslocamento de bactérias
            dx, dy = random.randint(-K, K), random.randint(-K, K)
            bacteria.move_bacteria(g, b, dx, dy)

            # Reprodução de bactérias
            if random.random() < 1 / TR:
                bacteria.reproduce(g, b, Q)

            # Fusão entre bactérias B e C
            if b.species == 'B' or b.species == 'C':
                if random.random() < 1 / TF:
                    for other_b in g[b.x][b.y]['bacteria']:
                        if other_b.species != b.species and other_b.species in ['B', 'C']:
                            # Criar bactéria BC e remover bactérias B e C
                            bc_bacteria = bacteria.create_bacteria('BC', F)
                            bc_bacteria.x, bc_bacteria.y = b.x, b.y
                            all_bacteria.append(bc_bacteria)
                            grid.add_bacteria_to_cell(g, b.x, b.y, bc_bacteria)
                            all_bacteria.remove(b)
                            all_bacteria.remove(other_b)
                            grid.remove_bacteria_from_cell(g, b.x, b.y, b)
                            grid.remove_bacteria_from_cell(g, b.x, b.y, other_b)
                            species_counts[b.species] -= 1
                            species_counts[other_b.species] -= 1
                            species_counts['BC'] += 1
                            break

            # Alimentação das bactérias
            if random.random() < 1 / TA:
                if b.species == 'A':
                    food_available = [other_b for other_b in g[b.x][b.y]['bacteria'] if other_b.species in ['B', 'C']]
                    if food_available:
                        target = random.choice(food_available)
                        all_bacteria.remove(target)
                        grid.remove_bacteria_from_cell(g, b.x, b.y, target)
                        species_counts[target.species] -= 1
                    elif b.food > 0:
                        bacteria.consume_food(b, 1)
                    else:
                        all_bacteria.remove(b)
                        grid.remove_bacteria_from_cell(g, b.x, b.y, b)
                        species_counts[b.species] -= 1
                else:
                    food_in_cell = g[b.x][b.y]['food']
                    if food_in_cell > 0:
                        grid.remove_food_from_cell(g, b.x, b.y, 1)
                    elif b.food > 0:
                        bacteria.consume_food(b, 1)
                    else:
                        all_bacteria.remove(b)
                        grid.remove_bacteria_from_cell(g, b.x, b.y, b)
                        species_counts[b.species] -= 1

            # Reposição de Alimento na Grelha
            if random.random() < 1 / TRP:
                grid.add_food_to_cell(g, b.x, b.y, 1)

            # Morte Natural
            if random.random() < 1 / TM:
                all_bacteria.remove(b)
                grid.remove_bacteria_from_cell(g, b.x, b.y, b)
                species_counts[b.species] -= 1

        t += 1
        events.append(species_counts.copy())

    # Gerar gráfico com evolução da quantidade de bactérias de cada espécie
    plot_evolution(events)

def plot_evolution(events):
    species = ['A', 'B', 'C', 'BC']
    colors = {'A': 'red', 'B': 'blue', 'C': 'green', 'BC': 'purple'}

    for s in species:
        plt.plot([e[s] for e in events], label=s, color=colors[s])

    plt.xlabel('Time')
    plt.ylabel('Bacteria count')
    plt.legend()
    plt.show()

# Exemplo de como executar a simulação com parâmetros definidos
run_simulation(N=8, Q=5, NA=10, NB=10, NC=10, F=5, TS=1000, TR=50, TF=50, TA=20, TRP=100, TM=200, K=1)

ValueError: list.remove(x): x not in list

In [4]:
import sys
sys.path.append("modulos")